# Explaination of Lab

# Uppgift 0 - setup

## 00-a Clone course repository
- First of all clone the LAB repository in your own github 

## 00-b Gitignore
- After creating the lab repositry in your github create the gitignore file and put following to ignore them during your lab project
- .wal
- .DS Store
- .dbeaver
- .project

## 00-c Changing folder name

- To perform this lab , we have alot of data in different schemas and we want to change the name of those schemas.
- For example from Datum 2024-08-13_2024-09-10 AIgineer and those schemas we have in lab in backend folder - raw data.
- We want to keep the schema name like Datum and rest remove it and copy them in clean data folder. 

### To perform this action we did some coding.

- Frist to all mention the path for raw data folder with the help of Pathlib and same for clean data folder 
- Create the clean folder so copy the renamed schemas  and this copying the schemas is happended with help of #copytree
- With the help of below block of code , change the raw schema into new schema name. 

```python
from pathlib import Path
from shutil import copytree, rmtree

raw_data_path = Path(__file__).parent / "raw_data"
cleaned_data_path = Path(__file__).parent / "cleaned_data"

if cleaned_data_path.is_dir():
    rmtree(cleaned_data_path)

cleaned_data_path.mkdir(parents=True, exist_ok=True)

for folder in raw_data_path.iterdir():
    new_name = folder.name.split()[0]
    copytree(folder, cleaned_data_path / new_name)
```


# 00-d Ingesting the CSV file in database with help of Python

- We have two CSV files in data folder and find where is the CSV file is with help of DATA_PATH.
- With the hep of setup_schema funtions set a schema with the name of supplier_invoice.
- Clean up the file name , create a table for those invoices and import the CSV data into the data.  

```python
def ingest_csv_data_to_duckdb():
    """ingesting the cleaned csv data to duckdb"""
    translation_table = str.maketrans({"å": "a", "ä": "a", "ö": "o"})

    for directory_path in CLEANED_DATA_PATH.glob("*"):
        schema_name = directory_path.name.lower().translate(translation_table)
        for csv_path in directory_path.glob("*"):
            table_name = csv_path.name.lower().split(".")[0].translate(translation_table)
```

## 00-e EDA on different schema

- Exploratry data analysist to better understand the data
- We explor different schema for exmaple datum and explor the two table in that schema like table and total.
- We fix the format of date column into Year-Month-Day.
- We have also join them to explor the data also see them in another angel. 

```sql
SELECT
    STRFTIME('%y-%m-%d', GEO_TOT.Datum) AS Datum,
    ANY_VALUE(GEO_TAB.Visningar) AS Visningar,
    ANY_VALUE(GEO_TAB."Visningstid (timmar)") AS "Visningstid (timmar)",
    GEO_TAB.Geografi
FROM  
    geografi.tabelldata AS geo_tab
CROSS JOIN
    geografi.totalt AS geo_tot
GROUP BY
    GEO_TAB.Geografi,
    STRFTIME('%y-%m-%d', GEO_TOT.Datum);
```


- Same way i did the EDA for geografi schema , run the different table in the schema to have good understand
- Did the Cross Join the two table in geografi schema and Group by different column to see on which Date , Geografi , Visningar , Visningar (timmar) and Genomsnittlig visningslängd 

```sql
SELECT * FROM  geografi.diagramdata d ;
   
SELECT  * FROM  geografi.tabelldata t ;
  
SELECT * FROM  geografi.totalt t ;

SELECT
    STRFTIME('%y-%m-%d', GEO_TOT.Datum) AS Datum,
    GEO_TAB.* 
FROM  
    geografi.tabelldata AS geo_tab
CROSS JOIN
    geografi.totalt  AS geo_tot
GROUP BY
    Datum, GEO_TAB.Geografi , GEO_TAB.Visningar , GEO_TAB."Visningstid (timmar)" , GEO_TAB."Genomsnittlig visningslängd" ;




# Uppgift 1 - plocka ut intressant data

## a) Create a Schema (Marts) . Your frontend should only pick data from the marts schedule and not from other schedules.

- Create a mart with name called marts_content.sql , in which first we create mart and then table with name content_veiw_time that the porpuse of this table is to show the views of each vidoe which is published on the ypoutbe channel or the database of youtube we have. 

```sql
CREATE SCHEMA IF NOT EXISTS marts;

CREATE TABLE IF NOT EXISTS marts.content_view_time AS 
(
SELECT
	Videotitel,
	"Publiceringstid för video" AS Publiceringstid,
	Visningar,
	"Visningstid (timmar)" AS Visningstid_timmar,
	Exponeringar,
	Prenumeranter,
	"Klickfrekvens för exponeringar (%)" AS "Klickfrekvens_exponering_%"
FROM
	innehall.tabelldata
ORDER BY
	"Visningstid (timmar)" DESC OFFSET 1);
```

- Now again in the same mart marts_content.sql . We have created one more table called views_per_date in which we see the number of views againist the date . 

```sql
CREATE TABLE IF NOT EXISTS marts.views_per_date AS (
SELECT
	STRFTIME('%Y-%m-%d',
	Datum) AS Datum,
	Visningar
FROM
	innehall.totalt);
```

- The same practice has been done for the next two more marts , i.e : marts_device.sql and marts_view_per_operativsystem. 

### mart_device.sql

- Let me tell me little about this mart , we have two table in this i.e: 
	
- device_view_date: 
	- In this table we show the number of views per Enhetyp i mean on which platform the video has been watch for example Surfplatta , Dator , TV , Mobiletelefon on same date.

```sql
CREATE TABLE IF NOT EXISTS marts.device_views_date AS (
SELECT
	STRFTIME('%Y-%m-%d',
	datum) AS Datum,
	Enhetstyp,
	sum(Visningar) AS Visningar
FROM
	enhetstyp.diagramdata d
GROUP BY
	(datum,
	Enhetstyp )
ORDER BY 
	Datum ASC );
```

- device_summary:
	- In this table we show the total numbers of views collective from each platform . 

```sql
CREATE TABLE IF NOT EXISTS marts.device_summary AS (
SELECT
	Enhetstyp,
	Visningar,
	"Visningstid (timmar)" AS Visningstid_timmar,
	"Genomsnittlig visningslängd" AS Visningslängd_genomsnitt
FROM
	enhetstyp.tabelldata );
```

### marts_view_per_operativesystem.sql

- This mart has also two two table i.e:

- operativesystem_per_view:
	- In this table we have tried to find out that how many views in total and how many views in total hour we get from different operative system per day.

```sql
CREATE TABLE IF NOT EXISTS marts.operativesystem_per_view AS
(
SELECT
    STRFTIME('%Y-%m-%d', optotal."Datum") AS Datum,
    optable."Operativsystem",
    SUM(optable."Visningar") AS Total_Visningar,
    SUM(optable."Visningstid (timmar)") AS Total_Visningstid_timmar
FROM  
    operativsystem.tabelldata AS optable
CROSS JOIN 
    operativsystem.totalt AS optotal
GROUP BY
    STRFTIME('%Y-%m-%d', optotal."Datum"),
    optable."Operativsystem"
ORDER BY
    STRFTIME('%Y-%m-%d', optotal."Datum"),
    optable."Operativsystem");
```

- views_per_geografi:
	- In this table we have tried to get the number of views in total againist the state per day. 
```sql
CREATE TABLE IF NOT EXISTS marts.views_per_geografi AS 
(
SELECT 
    STRFTIME('%y-%m-%d', geo_dia.Datum) AS Datum,
    GROUP_CONCAT(DISTINCT geo_dia.Geografi) AS Geografi,
    MAX(geo_tot.Visningar) AS Total_Viewers
FROM 
    geografi.diagramdata AS geo_dia
LEFT JOIN
    geografi.totalt AS geo_tot
ON
    geo_dia.Datum = geo_tot.Datum
GROUP BY 
    geo_dia.Datum
ORDER BY 
    geo_dia.Datum
);
```


# Uppgift 2 - skapa en frontend dashboard

## a) Run __main__.py alternatively streamlit run dashboard.py to run the streamlit dashboard

- This script is used to start a Streamlit dashboard. It finds the file called dashboard.py and runs it using Streamlit, which is a tool to build and run these web apps.

    - run_dashboard() function:
- This function finds the path of the dashboard.py file and then starts the dashboard by running Streamlit.
    - if __name__ == '__main__': block:
- This part ensures that the run_dashboard() function only runs when you directly run this script, not when it's imported somewhere else.

```python
import subprocess
from pathlib import Path

def run_dashboard():
    dashboard_path = Path(__file__).parent / "dashboard.py"
    subprocess.run(["streamlit", "run", dashboard_path])

if __name__ == '__main__':
    run_dashboard()
```


## b) Visualize more KPIs than the ones I included. Can be visualized with a graph or with text.

- I have did the KPI for mart in marts_view_per_operativesystem.sql for one table operativesystem_per_view in which we  see the 
    - Total number of Operative System with len() function.
    - To sum up the all viewers which we collect it from different operative system.
    - To sum up the viewers in hours and display it.

```python
class DeviceKPI:
    def __init__(self) -> None:
        self._device = QueryDatabase("SELECT * FROM marts.operativesystem_per_view").df

    def operative_views(self):
        df = self._device
        st.markdown("## KPI for Operative System")

        
        kpis = {
            "operative system" : len(df["Operativsystem"]),
            "Visningar": df["Total_Visningar"].sum(),
            "Visning Timmar": df["Total_Visningstid_timmar"].sum(),
        }

        
        for col, kpi in zip(st.columns(len(kpis)), kpis):
            with col:
                st.metric(kpi, round(kpis[kpi]))

```

- I have did the KPI for mart in marts_view_per_operativesystem.sql for one table views_per_geografi.sql in which we see.
    - The start date and end data of data we have.
    - Total views we get for same date for all states.


```python
class GeoKPI:
    def __init__(self):
        self._geografi = QueryDatabase("SELECT * FROM marts.views_per_geografi; ").df 

    def viewsPer_geografi(self):
        df = self._geografi
        st.markdown("## Viewers Per State")

        start_date = df["Datum"].min()
        end_date = df["Datum"].max()


        Kpis = {
            "Date" : f"{start_date} to {end_date}",
            "Total_Views" : df["Total_Viewers"].sum()
        }

        for col, kpi in zip(st.columns(len(Kpis)), Kpis):
            with col:
                st.metric(kpi, (Kpis[kpi]))


        st.dataframe(df)
```


## c) Create at least one filtering of the computer where the user can choose from a dropdown what to filter out. Here you can filter on graphs or metrics. 

- Mart in marts_view_per_operativesystem.sql for one table operativesystem_per_view. 
- Select DropDown
    - First of all , i made a selectdown box in which user can select the all operativesystem and use unique() function so it comes once. 
    - Save the selected option from operative system in filtered_df.
    - We get the first element for total_visnigar and for total_visningar_timmar 
    - Then we display them in with Total visinigar hiding and for Total visningar timamr we do the round to get the two digit after decimanl. 


```python
st.markdown("## KPI for Operative System in Selection down function ")

        selected_os = st.selectbox(
            "Select an Operating System",
            df["Operativsystem"].unique()
        )

        filtered_df = df[df["Operativsystem"] == selected_os]

        total_visningar = filtered_df["Total_Visningar"].values[0]
        total_visningstid_timmar = filtered_df["Total_Visningstid_timmar"].values[0]

        st.metric("Total Visningar", total_visningar)
        st.metric("Total Visningstid (Timmar)", round(total_visningstid_timmar, 2))
```

- Mart in marts_view_per_operativesystem.sql for one table views_per_geografi.sql.
- Select Dropdown
    - In this section we will get the KPI i.e Total viewer as per selected date
    - Same way as i explain above create the Select box to select the date
    - Save the selected date in variable filter_tab
    - We the get the total views as per that repective date.

```python
st.markdown("## Select down for to see how much total views on selected date")

        select_date = st.selectbox(
            "Select the date", df["Datum"]
        )

        filter_tab = df[df["Datum"] == select_date]

        total_viewers = filter_tab["Total_Viewers"].values[0]

        st.metric("Total_Viewers" , total_viewers)
```
